In [17]:

import sympy as sp
import numpy as np
from scipy.stats import ortho_group

# Skew-symmetric matrix function for cross product
def skew(vec_):
    """Returns the skew-symmetric matrix [vec]_x for a 3x1 vector."""
    vec = vec_.platten()
    return sp.Matrix([
        [0, -vec[2], vec[1]],
        [vec[2], 0, -vec[0]],
        [-vec[1], vec[0], 0]
    ])

def skew_np(vec_):
    vec = vec_.flatten()

    """Returns the skew-symmetric matrix [vec]_x for a 3x1 vector."""
    return np.array([
        [0, -vec[2], vec[1]],
        [vec[2], 0, -vec[0]],
        [-vec[1], vec[0], 0]
    ])

# Generate a random 3x3 orthogonal matrix
random_orthogonal_matrix = ortho_group.rvs(3)
random_orthogonal_matrix_2 = ortho_group.rvs(3)

print(random_orthogonal_matrix_2)

[[-0.54687543  0.05231756 -0.83557772]
 [-0.81673578  0.18605374  0.54619289]
 [ 0.18403784  0.9811457  -0.05901859]]


In [ ]:
import numpy as np
from scipy.spatial.transform import Rotation as R
from scipy.linalg import expm, logm


def h_acc(state, a_n, omega_n, i):
    R_imu, v_imu, p_imu, b_omega, b_a, R_c, p_c = state[:3], state[3:6].reshape(3, 1), \
        state[6:9].reshape(3, 1), state[9:12], state[12:15], state[15:18], state[18:21]
    if (i < 3):
        print("for the rotation part of SE2(3)\n")
        print(state[:3])
    R_c = expm(skew_np(state[15:18].reshape(3,))) @ random_orthogonal_matrix
    previous_chi = np.vstack((
                            np.hstack((random_orthogonal_matrix_2, v_imu, p_imu)),
                            np.hstack((np.zeros((2, 3)), np.eye(2)))
                        ))
    
    skew_5_5 = np.vstack((
        np.hstack((skew_np(state[15:18].reshape(3,)), np.zeros((3, 2)))),
        np.zeros((2, 5))
    ))
    
    R_imu = (expm(skew_5_5) @ previous_chi )[:3, :3]

    # R_imu = expm(skew(state[:3].reshape(3,))) @ random_orthogonal_matrix_2
    
    v_c = R_c.T @ (( R_imu.T @ v_imu).flatten() + np.cross(omega_n - b_omega, p_c))
    
    print("the second term SE2(3): \n",(R_c.T @  R_imu.T).T)
    # print("the B matrix phuc derivation is \n", (R_c.T @ skew (b_vector)).T)
    # print("the B_our matrix is \n", ((R_c.T @ skew(R_imu.T @ v_c))).T)
    # print("the R_imu.T is \n", R_imu.T)
    # print("the matrix C is \n", -skew(omega_n - b_omega))
    # print("FOR BIAS OMEGA \n", R_c.T @ skew(p_c))

    obs_paper = v_c
    return obs_paper

def compute_jacobian_paper(x_0, a_n, omega_n, d=1e-9):
    # Define small perturbations
    n = len(x_0)
    jacobian = np.zeros((3, n))

    # Compute the observation at the nominal state
    obs_nominal = h_acc(x_0, a_n, omega_n,10)

    for i in range(n):
        # Compute the observation with perturbed state
        x_0_perturbed = np.copy(x_0)
        x_0_perturbed[i] += d
        obs_perturbed = h_acc(x_0_perturbed, a_n, omega_n, i)
        # Calculate Jacobian element by finite difference
        # print(obs_perturbed)
        jacobian[:, i] = (obs_perturbed - obs_nominal) / d

    return jacobian

def h_acc_new(state, a_n, omega_n, i):
    R_imu1, v_imu, p_imu, b_omega, b_a, R_c, p_c = state[:3], state[3:6].reshape(3, 1), \
        state[6:9].reshape(3, 1), state[9:12], state[12:15], state[15:18], state[18:21]

    R_c = expm(skew_np(state[15:18].reshape(3,))) @ random_orthogonal_matrix
    previous_chi = np.vstack((
                            np.hstack((random_orthogonal_matrix_2, v_imu, p_imu)),
                            np.hstack((np.zeros((2, 3)), np.eye(2)))
                        ))
    skew_5_5 = np.vstack((
        np.hstack((skew_np(state[15:18].reshape(3,)), np.zeros((3, 2)))),
        np.zeros((2, 5))
    ))
    
    R_imu = (expm(skew_5_5) @ previous_chi )[:3, :3]
    v_c = R_c.T @ (( R_imu.T @ v_imu).flatten() + np.cross(omega_n - b_omega, p_c))

    b_vector = (( R_imu.T @ v_imu).flatten() + np.cross(omega_n - b_omega, p_c)).reshape(3,)
    omega_car = R_c.T @(omega_n - b_omega)
    a_car = R_c.T @ (a_n - b_a + np.cross(omega_n - b_omega, np.cross(omega_n - b_omega, p_c)))
    
    #  only print one time
    if(i == 0):
        ############ for check the derivation  ################
        M1 = np.array([0, 1, 0]).reshape(1, 3)
        M2 = np.array([1, 0, 0]).reshape(1, 3)
        M3 = np.array([0, 0, 1]).reshape(1, 3)

        alpha1 = (a_n - b_a + np.cross(omega_n - b_omega, np.cross(omega_n - b_omega, p_c))).reshape(3,1)
        alpha2 = (( R_imu.T @ v_imu).flatten() + np.cross(omega_n - b_omega, p_c)).reshape(3,1)
        alpha3 = (omega_n - b_omega).reshape(3,1)
        print(alpha1)
        
        # for term 2/7 correct
        # print("the 2/7 term : \n", - M2 @ (R_c.T @ R_imu.T) * (omega_car[2]))

        # for the  4/7 term not correct
        term_4_7 = -1 * -1 * M1 @ R_c.T @ (skew_np(omega_n - b_omega)) @ skew_np(p_c) \
                    + -1 * M1 @ R_c.T @ skew_np(skew_np(omega_n - b_omega) @ p_c) \
                    + (-1 * M2 @ R_c.T @ skew_np(p_c)) * (-omega_car[2]) \
                    + (-1 * M3 @ R_c.T) * (-v_c[0])
        
        # print("the 4/7 term : \n", term_4_7)

        #  for term 5/7 correct
        term_5_7 = -1 * M1 @ R_c.T 
        # print("the 5/7 term : \n", term_5_7)

        #  for term 6/7 not correct
        a = skew_np(alpha1)
        term_6_7 = M1 @ R_c.T @ skew_np(alpha1) \
                   + (M2 @ R_c.T @ skew_np(alpha2)) * (M3 @ R_c.T @ alpha3) \
                   + (M2 @ R_c.T @ alpha2) * (M3 @ R_c.T @ skew_np(alpha3))
        # print("the 6/7 term : \n", term_6_7)

        #  for term 7/7 correct
        term_7_7 = M1 @ R_c.T @ skew_np(omega_n - b_omega) @ skew_np(omega_n - b_omega) \
                    + M2 @ R_c.T @ skew_np( - omega_n + b_omega) * (omega_car[2])
        print("the 7/7 term : \n", term_7_7)

        # print("the B matrix phuc derivation is \n", (R_c.T @ skew (b_vector)).T)
        # print("the B_our matrix is \n", ((R_c.T @ skew(R_imu.T @ v_c))).T)
        # print("the R_imu.T is \n", R_imu.T)
        # print("the matrix C is \n", -skew(omega_n - b_omega))
        # print("FOR BIAS OMEGA \n", R_c.T @ skew(p_c))

    ############## find analytical solution #################
    # for SE2(3) term:
    obs_new = a_car[1] - v_c[0] * omega_car[2]
    return obs_new

def compute_jacobian_new_obser(x_0, a_n, omega_n, d=1e-10):
    # Define small perturbations
    n = len(x_0)
    jacobian = np.zeros((n))

    # Compute the observation at the nominal state
    obs_nominal = h_acc_new(x_0, a_n, omega_n, 2)

    for i in range(n):
        # Compute the observation with perturbed state
        x_0_perturbed = np.copy(x_0)
        x_0_perturbed[i] += d
        obs_perturbed = h_acc_new(x_0_perturbed, a_n, omega_n, i)
        # Calculate Jacobian element by finite difference
        # print(obs_perturbed)
        jacobian[i] = (obs_perturbed - obs_nominal) / d

    return jacobian

# Example usage:
chi_imu = np.array([0.0, 0.0, 0.0])
v_imu = np.array([0.5, 1.0, 3.0])
p_imu = np.array([0.3, 0.1, 0.8])
b_a = np.array([0.0, 0.0, 0.0])
b_omega = np.array([0.0, 0.0, 0.0])
chi_c = np.array([0.0, 0.0, 0.0])
p_c = np.array([0.1, 0.1, 0.1])

# Nominal state

a_n = np.array([1.0, 1.0, 1.0])
omega_n = np.array([0.5, 0.1, 1.0])
x_0 = np.concatenate((chi_imu, v_imu, p_imu, b_omega, b_a, chi_c, p_c))

In [32]:
# compute_jacobian_paper(x_0, a_n, omega_n).T
compute_jacobian_new_obser(x_0, a_n, omega_n).reshape(21, 1)

[[0.954]
 [0.89 ]
 [1.034]]
the 7/7 term : 
 [[-0.58127222 -0.18218422  0.30885453]]


array([[ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.1677436 ],
       [-0.23259172],
       [ 0.14151125],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.05122125],
       [-0.86621155],
       [-0.12387646],
       [-0.91077368],
       [-0.00556666],
       [-0.41286752],
       [ 0.42261084],
       [-0.57390315],
       [ 1.15942589],
       [-0.58127059],
       [-0.18218316],
       [ 0.30885516]])

In [20]:
from sympy import symbols, Matrix, cos, sin

# Define symbolic vectors for SE2(3)
chi_r = Matrix(symbols(r'chi_{r1} chi_{r2} chi_{r3}')).reshape(3, 1)
chi_v = Matrix(symbols(r'chi_{v1} chi_{v2} chi_{v3}')).reshape(3, 1)
chi_p = Matrix(symbols(r'chi_{p1} chi_{p2} chi_{p3}')).reshape(3, 1)

# Define the elements in the SE2(3) matrix
R = Matrix(3, 3, symbols('R1:10'))  # 3x3 rotation matrix
v = Matrix(3, 1, symbols('v1:4'))  # 3x1 velocity vector
p = Matrix(3, 1, symbols('p1:4'))  # 3x1 position vector

# Compute the inverse SE2(3) transformation
chi_inverse = Matrix.hstack(R.T, -R.T @ v, -R.T @ p)

# Define the full SE2(3) matrix
chi_epsilon = Matrix.vstack(chi_r, chi_v, chi_p)

# Define angle as the norm of the rotation vector (first 3 elements of state)
angle = (chi_r.norm())  # Use norm of chi_r for rotation
s = sin(angle)
c = cos(angle)

# Calculate a_ and b_
a_ = (1 - c) / angle
b_ = (angle - s / (angle**3))

# Corrected f_error expression
f_error = (- R.T @ chi_v +
           a_ * R.T @ (skew(chi_r) @ chi_v) +
        - b_ * R.T @ (skew(chi_r) @ (skew(chi_r) @ chi_v)))

# Jacobian of f_error w.r.t. chi_epsilon
Jacobian = f_error.jacobian(chi_epsilon)

f_error[:, 0]
# Correct substitution: Set all elements of chi_v and chi_p to 0
subs_dict = {var: 0.00 for var in chi_p}  # Set all chi_v elements to 0
# subs_dict.update({var: 0.00 for var in chi_v})  # Set all chi_p elements to 0
subs_dict.update({var: 0.00001 for var in chi_r})  # Set all chi_p elements to 0

# Apply substitution and simplify
Jacobian_simplified = Jacobian.subs(subs_dict)

# Print the final Jacobian
Jacobian_simplified

AttributeError: 'MutableDenseMatrix' object has no attribute 'platten'